## Summary

Use this notebook to delete lingering pipelines from GitLab.

---

## Imports

In [ ]:
import asyncio
import json
import os
import shlex
import subprocess

import aiohttp
from unsync import unsync

## Parameters

In [ ]:
PROJECT_ID = 21481523

In [ ]:
GITLAB_AUTH_TOKEN = os.environ["GITLAB_AUTH_TOKEN"]

In [ ]:
PIPELINES_TO_KEEP = [229764931]

## Workspace

The status of pipelines, one of: created, waiting_for_resource, preparing, pending, running, success, failed, canceled, skipped, manual, scheduled

<https://docs.gitlab.com/ee/api/pipelines.html#list-project-pipelines>.

In [ ]:
async def get_pipeline_infos(session):
    next_url = (
        f"https://gitlab.com/api/v4/projects/{PROJECT_ID}/pipelines?per_page=100"
    )
    pipeline_infos = []
    while next_url is not None:
        async with session.get(
            next_url, headers=[("PRIVATE-TOKEN", GITLAB_AUTH_TOKEN)]
        ) as response:
            try:
                next_url = response.links["next"]["url"]
            except KeyError:
                next_url = None
            pipeline_infos += await response.json()
    return pipeline_infos

In [ ]:
def filter_pipeline_infos(
    pipeline_infos,
    ids_to_exclude=PIPELINES_TO_KEEP,
    states_to_keep=("pending", "running", "success"),
):
    pipeline_infos = [
        pi
        for pi in pipeline_infos
        if pi["id"] not in ids_to_exclude and pi["status"] in states_to_keep
    ]
    return pipeline_infos

In [ ]:
async def retry_pipelines(session, pipeline_infos):
    for pipeline_info in pipeline_infos:
        url = (
            f"https://gitlab.com/api/v4/projects/{PROJECT_ID}/pipelines/{pipeline_info['id']}/retry"
        )
        async with session.post(
            url,
            headers=[("PRIVATE-TOKEN", GITLAB_AUTH_TOKEN)],
        ) as response:
            assert response.ok

In [ ]:
async def delete_pipelines(session, pipeline_infos):
    for pipeline_info in pipeline_infos:
        url = f"https://gitlab.com/api/v4/projects/{PROJECT_ID}/pipelines/{pipeline_info['id']}"
        async with session.delete(
            url,
            headers=[("PRIVATE-TOKEN", GITLAB_AUTH_TOKEN)],
        ) as response:
            assert response.ok

In [ ]:
# Remove succeeded pipelines
async with aiohttp.ClientSession() as session:
    pipeline_infos = await get_pipeline_infos(session)
    print(len(pipeline_infos), {p["status"] for p in pipeline_infos})
    pipeline_infos = filter_pipeline_infos(pipeline_infos, states_to_keep=("success"))
    print(len(pipeline_infos))
    await delete_pipelines(session, pipeline_infos)

In [ ]:
# Retry failed pipelines
async with aiohttp.ClientSession() as session:
    pipeline_infos = await get_pipeline_infos(session)
    print(len(pipeline_infos), {p["status"] for p in pipeline_infos})
    pipeline_infos = filter_pipeline_infos(pipeline_infos, states_to_keep=("failed"))
    print(len(pipeline_infos))
    await delete_pipelines(session, pipeline_infos)